In [26]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import EarlyStopping
import urllib.request as ur

In [27]:
# Load data
url = 'https://gml.noaa.gov/webdata/ccgg/trends/co2/co2_weekly_mlo.csv'
ur.urlretrieve(url,"co2_weekly_mlo.csv")
df = pd.read_csv("co2_weekly_mlo.csv", delim_whitespace=False, skiprows=35)
                 #names=["year", "month", "day", "decimal", "ppm", "days", "1year", "10year", "1800"])

In [28]:
df.head()

,year,month,day,decimal,average,ndays,1 year ago,10 years ago,increase since 1800
0,1974,5,19,1974.3795,333.37,5,-999.99,-999.99,50.39
1,1974,5,26,1974.3986,332.95,6,-999.99,-999.99,50.05
2,1974,6,2,1974.4178,332.35,5,-999.99,-999.99,49.59
3,1974,6,9,1974.4370,332.20,7,-999.99,-999.99,49.64
4,1974,6,16,1974.4562,332.37,7,-999.99,-999.99,50.06


In [29]:
df.tail()

,year,month,day,decimal,average,ndays,1 year ago,10 years ago,increase since 1800
2585,2023,12,3,2023.9219,421.02,2,419.23,396.43,142.07
2586,2023,12,10,2023.9411,422.20,7,418.81,396.39,142.96
2587,2023,12,17,2023.9603,422.24,5,419.05,397.72,142.74
2588,2023,12,24,2023.9795,421.86,5,419.41,397.53,142.12
2589,2023,12,31,2023.9986,422.52,4,419.34,397.73,142.55


In [30]:
df = df.rename(columns={"average" : "ppm"})

In [31]:
# Remove rows with ppm <= 0
df = df[df["ppm"] > 0]

# Train/val-test split
df_trainval, df_test = train_test_split(df, test_size=0.1, shuffle=False)
df_train, df_val = train_test_split(df_trainval, test_size=0.1, shuffle=False)

In [32]:

# Feature scaling
scaler = StandardScaler()
df_train_scaled = scaler.fit_transform(df_train["ppm"].values.reshape(-1, 1))
df_val_scaled = scaler.transform(df_val["ppm"].values.reshape(-1, 1))
df_test_scaled = scaler.transform(df_test["ppm"].values.reshape(-1, 1))

In [33]:
# Function to create sequences for LSTM
def create_sequences(data, seq_length):
    sequences = []
    for i in range(len(data) - seq_length):
        sequence = data[i:i + seq_length]
        target = data[i + seq_length]
        sequences.append((sequence, target))
    return np.array([s[0] for s in sequences]), np.array([s[1] for s in sequences])

In [34]:
# Define sequence length
seq_length = 10

In [35]:

# Create sequences for training, validation, and test data
X_train, y_train = create_sequences(df_train_scaled, seq_length)
X_val, y_val = create_sequences(df_val_scaled, seq_length)
X_test, y_test = create_sequences(df_test_scaled, seq_length)

In [36]:
# Build LSTM model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(seq_length, 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')

In [37]:
# Train the model
early_stopping = EarlyStopping(patience=5, restore_best_weights=True)
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val), callbacks=[early_stopping])

Epoch 1/50
65/65 [==============================] - 3s 11ms/step - loss: 0.4601 - val_loss: 0.5601
Epoch 2/50
65/65 [==============================] - 0s 5ms/step - loss: 0.0153 - val_loss: 0.1766
Epoch 3/50
65/65 [==============================] - 0s 5ms/step - loss: 0.0085 - val_loss: 0.0607
Epoch 4/50
65/65 [==============================] - 0s 5ms/step - loss: 0.0066 - val_loss: 0.0250
Epoch 5/50
65/65 [==============================] - 0s 5ms/step - loss: 0.0056 - val_loss: 0.0217
Epoch 6/50
65/65 [==============================] - 0s 5ms/step - loss: 0.0046 - val_loss: 0.0047
Epoch 7/50
65/65 [==============================] - 0s 5ms/step - loss: 0.0038 - val_loss: 0.0021
Epoch 8/50
65/65 [==============================] - 0s 5ms/step - loss: 0.0033 - val_loss: 0.0110
Epoch 9/50
65/65 [==============================] - 0s 5ms/step - loss: 0.0027 - val_loss: 0.0064
Epoch 10/50
65/65 [==============================] - 0s 5ms/step - loss: 0.0026 - val_loss: 0.0013
Epoch 11/50
65/65 

In [38]:
# Make predictions on validation and test data
y_val_pred = model.predict(X_val)
y_test_pred = model.predict(X_test)

8/8 [==============================] - 0s 2ms/step


In [39]:
# Inverse transform to get actual ppm values
y_val_pred_actual = scaler.inverse_transform(y_val_pred)
y_test_pred_actual = scaler.inverse_transform(y_test_pred)


In [40]:
# Evaluate performance on validation and test data
mae_val = mean_absolute_error(df_val["ppm"].iloc[seq_length:], y_val_pred_actual)
mae_test = mean_absolute_error(df_test["ppm"].iloc[seq_length:], y_test_pred_actual)

In [41]:
# Visualize results
fig = go.Figure()

# Plot train/val, validation, and test data
fig.add_scatter(x=df_trainval["decimal"], y=df_trainval["ppm"], name="Train/Val Data")
fig.add_scatter(x=df_val["decimal"].iloc[seq_length:], y=df_val["ppm"].iloc[seq_length:], name="Validation Data")
fig.add_scatter(x=df_test["decimal"].iloc[seq_length:], y=df_test["ppm"].iloc[seq_length:], name="Test Data")

# Plot fit for train/val data and val fit
fig.add_scatter(x=df_val["decimal"].iloc[seq_length:], y=y_val_pred_actual.flatten(), name="Validation Prediction")
fig.add_scatter(x=df_test["decimal"].iloc[seq_length:], y=y_test_pred_actual.flatten(), name="Test Prediction")

# Update the x/y axis labels
fig.update_xaxes(title_text="Time [Years]")
fig.update_yaxes(title_text="Mauna Loa CO2 Conc Measurement [ppm]")

fig.show()

# Display MAE
print(f'MAE on Validation Data: {mae_val:.4f}')
print(f'MAE on Test Data: {mae_test:.4f}')

MAE on Validation Data: 0.5585
MAE on Test Data: 0.6233


In [43]:
# Display MAE
print(f'MAE on Validation Data: {mae_val:.4f}')
print(f'MAE on Test Data: {mae_test:.4f}')

MAE on Validation Data: 0.5585
MAE on Test Data: 0.6233
